# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [1]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

170508288/170498071 [==============================] - 4s 0us/step


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10_1/v6/1.1.0.incomplete96DCRM/cifar10_1-test.tfrecord


  0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset cifar10_1 downloaded and prepared to /root/tensorflow_datasets/cifar10_1/v6/1.1.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


# **2. 데이터 전처리**



In [2]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test1_after = x_test1 / 255.0
x_test2_after = x_test2 / 255.0

In [3]:
import numpy as np
print('x_train mean : ', np.mean(x_train))
print('x_test1 mean : ', np.mean(x_test1))
print('x_test2 mean : ', np.mean(x_test2))

print('x_train std : ', np.std(x_train))
print('x_test1 std : ', np.std(x_test1))
print('x_test2 std : ', np.std(x_test2))

x_train mean :  120.70756512369792
x_test1 mean :  121.52915475260417
x_test2 mean :  121.93687776692708
x_train std :  64.1500758911213
x_test1 std :  64.06097012299574
x_test2 std :  65.75933956009034


In [4]:
# x_train의 mean과 std를 구하여 normalization 진행
# x_train, x_test1, x_test2를 x_train의 mean, std를 이용하여 normalization

x_train_after = (x_train-np.mean(x_train))/np.std(x_train)
x_test1_after = (x_test1 - np.mean(x_train))/np.std(x_train)
x_test2_after = (x_test2-np.mean(x_train))/np.std(x_train)

In [5]:
print("=== x_train_after : Mean, Std ===")
print(np.mean(x_train_after))
print(np.std(x_train_after))

print("\n=== x_test1_after : Mean, Std ===")
print(np.mean(x_test1_after))
print(np.std(x_test1_after))

print("\n=== x_train_after : Mean, Std ===")
print(np.mean(x_test2_after))
print(np.std(x_test2_after))

=== x_train_after : Mean, Std ===
-2.5247951877342226e-17
1.0000000000000022

=== x_test1_after : Mean, Std ===
0.012807305642173993
0.9986109795368484

=== x_train_after : Mean, Std ===
0.01916307387251756
1.0250859199558913


In [6]:
print('x_train_after : ', x_train_after.shape)
print('x_test1_after : ', x_test1_after.shape)
print('x_test2_after : ', x_test2_after.shape)

x_train_after :  (50000, 32, 32, 3)
x_test1_after :  (10000, 32, 32, 3)
x_test2_after :  (2000, 32, 32, 3)


# Data Augmentation

In [7]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Code for Cutout Augmentation
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [9]:
image_generator = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.10,
    shear_range=0.5,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    preprocessing_function = get_random_eraser(v_l=0, v_h=255),
)

In [10]:
image_generator.fit(x_train_after, augment=True)

train_size = x_train_after.shape[0]
augment_size=100000
# x_train_after에서 random으로 뽑아와서 해당 데이터를 augmentation 진행
idx = np.random.randint(train_size, size=augment_size)
x_augmented = x_train_after[idx].copy()
y_augmented = y_train[idx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size), batch_size=augment_size, shuffle=False).next()[0]

In [11]:
print('x_augmented shape : ', x_augmented.shape)
print('y_augmented shape : ', y_augmented.shape)

x_augmented shape :  (100000, 32, 32, 3)
y_augmented shape :  (100000, 10)


In [12]:
# 기존 데이터와 추가된 이미지 합치기
x_train_after = np.concatenate((x_train_after, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

In [13]:
print(x_train_after.shape)
print(y_train.shape)

(150000, 32, 32, 3)
(150000, 10)


# **3. 모델 생성**

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, LeakyReLU, Activation
from keras.layers import BatchNormalization
# from keras.layers.advanced_activations import LeakyReLU


model = Sequential()

# model.add(Conv2D(32, (3, 3), input_shape = input_shape))
# model.add(Activation('relu'))
model.add(Conv2D(64, kernel_initializer='glorot_uniform', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.2))

# model.add(Conv2D(32,(3,3)))
# model.add(Activation('relu'))
model.add(Conv2D(64, kernel_initializer='glorot_uniform', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.2))

# model.add(Conv2D(64, (2, 2)))
# model.add(Activation('relu'))
model.add(Conv2D(128, kernel_initializer='glorot_uniform', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.2))

# model.add(Conv2D(64, (3, 3)))
model.add(Conv2D(128, kernel_initializer='glorot_uniform', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.2))

# model.add(Conv2D(128, (3, 3)))
model.add(Conv2D(256, kernel_initializer='glorot_uniform', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.2))

# model.add(Conv2D(128, (3, 3)))
model.add(Conv2D(256, kernel_initializer='glorot_uniform', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(LeakyReLU(alpha=0.01))
# model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
# model.add(Activation('relu'))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping

# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',  # 모니터 기준 설정 (val loss) 
                              patience=10,         # 10회 Epoch동안 개선되지 않는다면 종료
                             )
# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set (x_test1_after, x_test2_after, 혹은 둘을 merge해서 사용)
model.fit(x_train_after, y_train, batch_size = 128, epochs = 100, shuffle=True, callbacks=[cp_callback], validation_data=(x_test2_after, y_test2))

Epoch 1/100
1172/1172 [==============================] - 48s 14ms/step - loss: 2.3108 - accuracy: 0.0997 - val_loss: 2.3027 - val_accuracy: 0.1000

Epoch 00001: val_accuracy improved from -inf to 0.10000, saving model to /content/checkpoint_entire_best.h5
Epoch 2/100
1172/1172 [==============================] - 15s 13ms/step - loss: 2.2455 - accuracy: 0.1313 - val_loss: 2.0764 - val_accuracy: 0.2010

Epoch 00002: val_accuracy improved from 0.10000 to 0.20100, saving model to /content/checkpoint_entire_best.h5
Epoch 3/100
1172/1172 [==============================] - 15s 13ms/step - loss: 1.7153 - accuracy: 0.3445 - val_loss: 1.6246 - val_accuracy: 0.4035

Epoch 00003: val_accuracy improved from 0.20100 to 0.40350, saving model to /content/checkpoint_entire_best.h5
Epoch 4/100
1172/1172 [==============================] - 15s 13ms/step - loss: 1.3860 - accuracy: 0.5020 - val_loss: 1.4934 - val_accuracy: 0.4640

Epoch 00004: val_accuracy improved from 0.40350 to 0.46400, saving model to /c

In [ ]:
'''
# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Flatten layer: 32 x 32 x 3 image를 3072개의 1D vector input으로 변환
model.add(keras.layers.Flatten(input_shape=input_shape))

# 1st hidden layer: fully-connected layer
# (# of inputs = 3072, # of outputs = 512, actication fuction = relu)
model.add(keras.layers.Dense(512, activation=tf.nn.relu))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 512, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(256, activation=tf.nn.relu))

# 3rd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 64, actication fuction = relu)
model.add(keras.layers.Dense(64, activation=tf.nn.relu))

# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 10, actication fuction = softmax)
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set (x_test1_after, x_test2_after, 혹은 둘을 merge해서 사용)
model.fit(x_train_after, y_train, batch_size = 128, epochs = 5, shuffle=True, callbacks=[cp_callback], validation_data=(x_test2_after, y_test2))
'''

# **4. 모델 저장**

In [19]:
save_path = '/content/'
team_name = 'team10'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [20]:
save_path = '/content/'
team_name = 'team10'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test1_after, y_test1)
model.evaluate(x_test2_after, y_test2)

63/63 [==============================] - 0s 3ms/step - loss: 0.8946 - accuracy: 0.7065


[0.894612193107605, 0.7064999938011169]